In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests


## Import sextant prices from tft

In [2]:
compass_prices = requests.get('https://github.com/The-Forbidden-Trove/tft-data-prices/blob/master/lsc/bulk-compasses.json')

In [3]:
compass_prices_json = json.loads(json.loads(compass_prices.text)['payload']['blob']['rawLines'][0])['data']

In [4]:
sextant_name_mappings = requests.get('https://github.com/The-Forbidden-Trove/tft-data-prices/blob/master/mappings/compasses.json')

In [5]:
sextant_mapping_dict = json.loads(''.join(json.loads(sextant_name_mappings.text)['payload']['blob']['rawLines']).replace('Sextant ', ''))

sextant_mapping_dict = {k.replace(" ","").lower(): v for k,v in sextant_mapping_dict.items()}
sextant_mapping_dict['playersgainanadditionalvaalsoulonkillyourmapscontain6additionalpacksofcorruptedvaalmonsters'] = 'Vaal Soul On Kill'

## Getting sextant weights form poedb.tw

In [6]:
sextant_weights = requests.get('https://poedb.tw/us/Awakened_Sextant#Sextant')

In [7]:
sextant_weights_bs4 = BeautifulSoup(sextant_weights.text, 'html.parser')

In [8]:
sextant_weights_tbody = sextant_weights_bs4.find('div', id='Sextant').tbody


In [9]:
sextant_dict = {}

for tr in sextant_weights_tbody:
    sextant_info = []
    for td in tr:
        sextant_info.append(td.text)
    sextant_info[2] = sextant_info[2].split('3 uses remaining')[0].replace(' ','').lower()
    
    sextant_info[2] = sextant_info[2].replace('arehauntedby', 'contain')
    sextant_info[2] = sextant_info[2].replace('mapbossesdropanadditionaluniqueitem', 'mapbossesdrop1additionaluniqueitem')
    sextant_info[2] = sextant_info[2].replace('mapbossesdeal100%moredamagemapbosseshave200%morelifeanadditionalshaperguardianmapdropsoncompletingyourmaps(tier14+)',
                                      'finalmapbossineachmapdropsanadditionalshaperguardianmap(tier14+)mapbosseshave200%morelifemapbossesdeal100%moredamage')
    sextant_info[2] = sextant_info[2].replace('mapbossesdeal100%moredamagemapbosseshave200%morelifeanadditionalelderguardianmapdropsoncompletingyourmaps(tier14+)',
                                     'finalmapbossineachmapdropsanadditionalelderguardianmap(tier14+)mapbosseshave200%morelifemapbossesdeal100%moredamage')
    sextant_info[2] = sextant_info[2].replace('mapbossesdeal100%moredamagemapbosseshave200%morelifeanadditionalconquerormapdropsoncompletingyourmaps(tier14+)',
                                     'finalmapbossineachmapdropsanadditionalconquerormapmapbosseshave200%morelifemapbossesdeal100%moredamage')
    sextant_info[2] = sextant_info[2].replace('itemsfoundinyouridentifiedmapsareidentified20%increasedpacksizeinyourunidentifiedmaps',
                                     '20%increasedpacksizeinyourunidentifiedmaps')
    sextant_info[2] = sextant_info[2].replace('yourmapscontain6additionalpacksofcorruptedvaalmonstersyourmapshavea50%chancetocontaingiftsoftheredqueenpermortalfragmentusedyourmapshavea50%chancetocontaingiftsofthesacrificedpersacrificefragmentused',
                                     'yourmapscontain6additionalpacksofcorruptedvaalmonstersyourmapshavea50%chancetocontaingiftsofthesacrificedpersacrificefragmentusedyourmapshavea50%chancetocontaingiftsoftheredqueenpermortalfragmentused')
    
    
    
    sextant_info[3] = sextant_info[3].split('default ')[1]
    sextant_dict[sextant_info[2]] =  {'weight': sextant_info[3]}

for sextant in sextant_mapping_dict:
    if sextant in sextant_dict:
        sextant_dict[sextant]['name'] = sextant_mapping_dict[sextant]
        sextant_dict[sextant]['price'] = [compass_price['chaos'] for compass_price in compass_prices_json if compass_price['name'] == sextant_dict[sextant]['name']][0]



## Get Sextant Price from poe.ninja

In [10]:
currency_price = requests.get('https://poe.ninja/api/data/currencyoverview?league=Ancestor&type=Currency')

In [11]:
currency_price_json = json.loads(currency_price.text)

In [12]:
for currency in currency_price_json['lines']:
    if currency['currencyTypeName'] == 'Awakened Sextant':
        Sextant_Price = currency['chaosEquivalent']


In [13]:
df = pd.DataFrame.from_dict(sextant_dict, orient = 'index')
df.reset_index(inplace=True, drop=True)
df = df[['name','weight','price']]
df["weight"] = df["weight"].apply(int)
df["price"] = df["price"].apply(int)
df.sort_values('price', ascending=False,inplace=True)
df.reset_index(inplace=True, drop=True)
pd.set_option('display.max_rows', None)
df



,name,weight,price
0,8 Modifiers,50,152
1,Conqueror Map,50,150
2,Strongbox Enraged,250,135
3,Yellow Plants,50,90
4,Purple Plants,50,60
5,Blue Plants,50,60
6,Delirium Reward,50,60
7,Beyond,250,55
8,Sacred Grove,100,54
9,Shaper Guardian,50,35


In [14]:
#assume every sextant 10c+ sells and nothing below that
#assume sextant blocking

revenue_per_sex = sum(df[df['price']>=10]['weight']*df[df['price']>=10]['price'])/(sum(df['weight'])-4000)

profit_per_sex = revenue_per_sex - Sextant_Price - 1 

print(profit_per_sex)

1.520731513384575


In [15]:
#df.to_excel('rolling sex.xlsx')